In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.set_option('display.max_columns', None)

ckd_df = pd.read_excel(
    "Data/ckd_dataset.xlsx",
    usecols = "B:Y,AL:AQ,AS:AY"
)

#filtering
def ckd_stage_clf(eGFR):
    if eGFR >= 90:
        return 1 #there is actually no stage 1 data
    elif eGFR >= 60:
        return 2
    elif eGFR >= 45:
        return 3 #3a
    elif eGFR >= 30:
        return 4 #3b
    elif eGFR >= 15:
        return 5
    else:
        return 6


ckd_df.dropna(inplace=True)
eGFR_final_mean = ckd_df["eGFR(last visit)"].mean()
eGFR_final_std = ckd_df["eGFR(last visit)"].std()
filtered_ckd_df = ckd_df[(ckd_df["eGFR(last visit)"] < (eGFR_final_mean + 4*eGFR_final_std)) & (ckd_df["eGFR(last visit)"] < (.95 * ckd_df["eGFR"]))]
filtered_ckd_df.loc[:, "CKD_stage"] = filtered_ckd_df.loc[:, "eGFR"].apply(ckd_stage_clf)

filtered_ckd_df

,gender,age,SBP,BMI,etiology of CKD,Hb,Alb,Cr,eGFR,CKD_stage,CKD category,dip-stick proteinuria,proteinuria,urinary occult blood,UPCR,UPCR category,hypertension,prevalence of CVD,diabetes,use of RAASi,use of CCB,use of diuretics,eGFR(0M),50%eGFR,eGFR(last visit),observational duration,50%eGFR_reached,50%eGFR_duration,RRT,RRT_duration,CKD progression,CKD progression_duration,death,death_duration,development of CVD,development of CVD_duration,≥6M followed
0,2,74,120.0,23.137669,2,12.0,4.0,1.20,34.146986,4,6.0,2.0,1.0,0.0,1.253731,3.0,1,0,0,1,1,0,34.146986,17.073493,18.495328,37,0,37,0,37,0,37,0,37,0,37,1
2,1,74,143.0,24.554564,2,10.9,3.8,4.95,9.804696,6,12.0,3.0,1.0,0.0,1.759615,3.0,1,0,1,1,1,1,9.804696,4.902348,5.578057,13,0,13,1,13,1,13,0,13,0,13,1
3,1,57,139.0,28.515625,2,15.9,4.8,0.84,73.570568,2,4.0,1.0,1.0,0.0,0.513514,3.0,1,0,0,0,0,0,73.570568,36.785284,67.225032,37,0,37,0,37,0,37,0,37,0,37,1
4,1,32,154.0,24.582701,4,14.4,4.4,0.87,83.558305,2,1.0,-1.0,0.0,0.0,0.025000,1.0,1,0,0,0,0,0,83.558305,41.779153,69.171408,36,0,36,0,36,0,36,0,36,0,36,1
7,1,66,148.0,22.532285,1,10.2,3.3,5.39,9.230697,6,12.0,3.0,1.0,0.0,3.535088,3.0,1,0,1,0,1,1,9.230697,4.615349,8.403298,6,0,6,1,8,1,8,0,8,0,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,1,75,120.0,26.015635,1,11.2,3.9,2.56,20.092993,5,8.0,-1.0,0.0,0.0,0.210210,2.0,1,0,1,1,1,0,20.092993,10.046496,15.511510,37,0,37,0,37,0,37,0,37,0,37,1
1128,2,72,169.0,22.494988,1,6.6,2.7,3.12,12.100035,6,12.0,3.0,1.0,0.0,17.298091,3.0,1,0,1,0,1,0,12.100035,6.050017,9.285258,7,0,7,1,10,1,10,0,10,0,7,1
1131,1,71,154.0,24.238775,2,15.4,4.2,1.08,52.471683,3,6.0,2.0,1.0,0.0,0.760234,3.0,1,0,0,1,0,0,52.471683,26.235842,47.952934,38,0,38,0,38,0,38,0,38,0,38,1
1132,1,81,154.0,19.596458,2,9.0,3.5,2.00,25.747776,5,9.0,2.0,1.0,0.0,1.038526,3.0,1,1,0,1,1,1,25.747776,12.873888,17.923736,37,0,37,0,37,0,37,0,37,0,37,1
